# Machine Learning Project

*by Tom Cavey, May 9, 2018*

## Introduction

In my project, I will try to predict the vehicle manufactuerer of a car pictured. I am using a Transfer learning model for image classification from Google called Inception-V3. This pre-trained model is not specific to recognizing cars, but I hope to test many pre-trained networks and compare them.

Transfer learning is where you take a neural network that has already been trained for a different task, and tweak it to solve a new problem or task. Because I want to modify the network to meet the needs of my dataset, the last layer (or more) of the pre-trained network will be removed and replaced with my own classifier. It is important that the weights of the layers that preceed the last one are not changed.

This is much easier than building a network from scratch, like I originally begun to do (See "Things that went wrong" section). With Transfer learning the key to making the pre-trained model work with a new dataset is to fine-tune the model. This can be tricky, because if the previous task that the network was originally trained and used for is too different than the problem I am trying to solve, it could result in not being able to properly classify the data.
In building the new model with Transfer learning since most of it is made already, you can train your model using fewer computational resources and a reduction training time.

It is obvious why using a Transfer learning network is much easier than building a new network from scratch. However, just because it is easier does not mean that it will be more accurate. This is the downside of using a Trasnfer learning network that has been pre-trained on other data. So, keeping this in mind, and not being able to successfully build a convolutional neural net from scratch- I will set my goals to try and test multiple different Transfer learning networks and compare them.

## The Vehicle Data set

First I wanted to find the proper data set. Initially, all that seemed to be available were low quality images of the rear end of some vehicles which were found in some computer vision Github repositories. This wouldn't do because I needed images that were from all sides fo the vehicles, and in high resolution. (It wasn't until later that I realized raw high res images probably arent the best suited for me). I did find a Cifar dataset that seemed OK, but I later stumbled upon a webpage from Jonathan Krause, a Computer Science student at Standford University who did a similar experiment to mine- where all the images had been made available to download. To my suprise, this data set had nice high resolution images of all makes of cars. Unfortuneatly they weren't yet divided into classification folders. I had to do this manually. So I went through the 16,000 image database and seperated the cars by manufactuerer into classification folders. Turned out to be fairly simple because they were ordered in descending alphabetical order by manufactuerer (thank you, Jonathan.)

Once I got the images in their seperate classification folders, I created the testing image folder. I went through each of the manufactueres and selected some cars from the original train batch. I had planned to use some of my own pictures I took at car meets and pictures of my own car. I had to make sure I did not include pictures of cars that were not in the training set, but I thought it would be funny to see what a Subaru would be identified as, which is not in my original training set. (more on this later).

# Methods

### Setting up the Transfer Learning Network
First thing I had to do was set up the Transfer Learning network to use my data set. This was not as straight forward as I might make it seem, but after playing around with it for a few days I finally figured it out. This led me to realize that I needed to resize my images.

### Resizing the images
Tensorflow needs to have the images be resized to a 299 x 299 square. The original code in the IBM PowerAI Transfer learning Github had a method which took care of this resizing for me. It takes the raw images from my directory with all the classification folders and resizes them into a new directory, cool!

#### For example, this was a raw data image (full res):

<img src="015846.jpg">

#### To this 299 x 299 image: 

<img src="sample_images/smallC30.jpg">

Using the following command, I was able to install the needed Python package to resize the images. The cell below only needs to be run once, and the notebook kernel needs to be restarted.

### Organizing the images by classification
This part I had to do manually, moving each batch of vehicle into its correct folder. The CS445_images folder contains jpeg images in a directory of sub folders with each labeled as "Acura", "Volvo", "Ferrari", etc. The subfolder names are crucial because that's how TensorFlow knows which classification label belongs to which image, it uses the subfolder's name as the classification label for the images which are contained within.

In [1]:
#This was used to install the package necessary to resize the images.
#!pip install --user python-resize-image==1.1.11

### Imports
Import statements. Taken directly from the IBM PowerAI Github repo. (note: no PowerAI was used)

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
import os.path
import shutil
import sys

from PIL import Image
from resizeimage import resizeimage
import tensorflow as tf
from tensorflow.python.framework import graph_util
from tensorflow.python.platform import gfile

module_path = os.path.abspath('..')
if module_path not in sys.path:
    sys.path.append(module_path)

from image_retraining import retrain

/Users/Boulderc30/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Prepare the images
The Inception model requires 299 X 299 pixel sizes.
First copy the files from `stored_images_resized` into `images_resized_dir`.
With these stored images that are already resized, we don't need to repeat the process.
Next copy and resize the remaining raw images from `image_dir` into `images_resized_dir`.

We don't need to use any of the code below, as I already have done this step in the process. I commented it out because I would like to continue using it later.

In [3]:
# def resize_images(src_dir, dest_dir):
#     if not os.path.isdir(src_dir):
#         raise Exception(src_dir + " is not a directory")
#     if not os.path.exists(dest_dir):
#         os.mkdir(dest_dir)

#     raw_images = {image for image in os.listdir(src_dir) if image.endswith(
#         JPEG_EXTENSIONS)}
#     dest_images = {image for image in os.listdir(dest_dir)}

#     # Resize the ones that are not already in the dest dir
#     for image in raw_images - dest_images:
#         if DEBUG:
#             print("Resizing " + image)
#         resize_image(image, src_dir, dest_dir)

# def resize_image(image_file, src_dir, dest_dir):
#     in_file = os.path.join(src_dir, image_file)
#     with open(in_file, 'r+b') as fd_img:
#         with Image.open(fd_img) as img:
#             resized_image = resizeimage.resize_contain(
#                 img, [299, 299]).convert("RGB")
#             resized_image.save(os.path.join(dest_dir, image_file), img.format)

# # Use a fresh working dir for the resized images
# if os.path.isdir(images_resized_dir):
#     shutil.rmtree(images_resized_dir)
# os.mkdir(images_resized_dir)
    
# subdirs = ('Acura', 'Aston_Martin', 'Audi', 'Bentley'
#           , 'BMW', 'Bugatti', 'Buick', 'Cadillac', 'Chevy', 'Chrystler'
#           , 'Daewoo', 'Dodge', 'Ferrari', 'Fiat', 'Fisker', 'Ford'
#            , 'Geo', 'GMC', 'Honda', 'Hummer', 'Hyundai', 'Infiniti'
#           , 'Isuzu', 'Jaguar', 'Jeep', 'Lamborghini', 'Land_Rover'
#           , 'Lincoln', 'Maybach', 'Mazda', 'Mclaren', 'Mercedes_Benz'
#           , 'Mini', 'Mitsubishi', 'Nissan', 'Porsche', 'Rolls_Royce'
#           , 'Scion', 'Smart', 'Spyker', 'Suzuki', 'Tesla', 'Toyota'
#           , 'Volkswagen', 'Volvo')

# # Copy in the image files, 'Suzuki'
# for subdir in subdirs:
#     dest_dir = os.path.join(images_resized_dir, subdir)
#     if not os.path.isdir(dest_dir):
#         os.mkdir(dest_dir)
      
#     # Copy the already resized files first, if any, from the repo or a custom dir
#     if stored_images_resized:
#         source_dir = os.path.join(stored_images_resized, subdir)
#         if os.path.isdir(source_dir):
#             for f in os.listdir(source_dir):
#                 path = os.path.join(source_dir, f)
#                 if (os.path.isfile(path)):
#                     shutil.copy(path, dest_dir)
                    
#     # Copy/resize the remaining raw images into the images_resized_dir(s)
#     resize_images(os.path.join(image_dir, subdir), dest_dir)

Below is the bottleneck producing code for the supplied images. This is commented out as it only needs to calculate the bottlenecks once (unless I add more images). This takes a really long time to run, don't run it unless you really mean to!

In [4]:
# # Use a fresh working dir for the bottleneck files  
# if os.path.isdir(bottleneck_dir):    
#     shutil.rmtree(bottleneck_dir)
# os.mkdir(bottleneck_dir)

# subdirs = ('Acura', 'Aston_Martin', 'Audi', 'Bentley'
#           , 'BMW', 'Bugatti', 'Buick', 'Cadillac', 'Chevy', 'Chrystler'
#           , 'Daewoo', 'Dodge', 'Ferrari', 'Fiat', 'Fisker', 'Ford'
#            , 'Geo', 'GMC', 'Honda', 'Hummer', 'Hyundai', 'Infiniti'
#           , 'Isuzu', 'Jaguar', 'Jeep', 'Lamborghini', 'Land_Rover'
#           , 'Lincoln', 'Maybach', 'Mazda', 'Mclaren', 'Mercedes_Benz'
#           , 'Mini', 'Mitsubishi', 'Nissan', 'Porsche', 'Rolls_Royce'
#           , 'Scion', 'Smart', 'Spyker', 'Suzuki', 'Tesla', 'Toyota'
#           , 'Volkswagen', 'Volvo')

# # Copy in the stored bottleneck files
# for subdir in subdirs:
#     dest_dir = os.path.join(bottleneck_dir, subdir)
#     if not os.path.isdir(dest_dir):
#         os.mkdir(dest_dir)

#     image_dest_dir = os.path.join(images_resized_dir, subdir)

#     if stored_bottlenecks:
#         source_dir = os.path.join(stored_bottlenecks, subdir)
#         if os.path.isdir(source_dir):
#             for f in os.listdir(source_dir):
#                 path = os.path.join(source_dir, f)
#                 if (os.path.isfile(path)):
#                     # Copy the persisted bottleneck to bottlenecks dir
#                     shutil.copy(path, dest_dir)
#                     # "touch" the file (w/o the .txt) to create a placeholder image
#                     # This image file will only be used to build the lists.
#                     if DEBUG:
#                         print("Creating placeholder image at %s" % os.path.join(image_dest_dir, f[:-4]))
#                     open(os.path.join(image_dest_dir, f[:-4]), 'a').close
                        

### Parameters

This is where I changed the parameters from the old Transfer learning network, to use my data set. I removed the original image_dir (the one which contained all the raw images) because the file was roughly 1.99GB in size. The compressed and resized version of the image_dir is much smaller.

In [5]:
# Set DEBUG to True for more output
DEBUG = False

# Expect image files to always end with one of these
JPEG_EXTENSIONS = ('.jpeg', '.JPEG', '.jpg', '.JPG')

# Raw input images come from this dir in the git repo (or you can customize this to point to a new dir).
# Only JPEG images are used. We will resize these images before using them.
#image_dir = '../data/CS445_images'

# We kept some images separate for our manual testing at the end.
test_images_dir = 'newTest'

# If stored_images_resized, images here have already been resized are can be used w/o re-resizing
stored_images_resized = 'images_resized'  # set to None to ignore

# If stored_bottlenecks, supplement the image_dir collection with persisted bottlenecks from this dir
stored_bottlenecks = 'bottlenecks'  # set to None to ignore

# Working files are in /tmp by default
# tmp_dir = '/tmp'
bottleneck_dir = os.path.join('bottlenecks')
images_resized_dir = os.path.join('images_resized')
summaries_dir = os.path.join('retrain_logs')

# Download the original inception model to/from here
model_dir = os.path.join('inception')
inception_url = 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'

# Store the graph before and after training
output_graph_orig = "output_graph_orig.pb"
output_graph = "output_graph.pb"
output_labels = "output_labels.txt"

#Total number of misclassified images
total = 0

# Training params
architecture = 'inception_v3'
final_tensor_name = "final_result"
how_many_training_steps = 700
learning_rate = 0.2
testing_percentage = 20
validation_percentage = 80
eval_step_interval = 10
train_batch_size = 100
test_batch_size = 200
validation_batch_size = 100
print_misclassified_test_images = True

# Create a FLAGS object with these attributes
FLAGS = type('FlagsObject', (object,), {
    'architecture': architecture,
    'model_dir': model_dir,
    'intermediate_store_frequency': 0,
    'summaries_dir': summaries_dir,
    'learning_rate': learning_rate,
    'image_dir': images_resized_dir,
    'testing_percentage': testing_percentage,
    'validation_percentage': validation_percentage,
    'bottleneck_dir': bottleneck_dir,
    'final_tensor_name': final_tensor_name,
    'how_many_training_steps': how_many_training_steps,
    'train_batch_size': train_batch_size,
    'test_batch_size': test_batch_size,
    'eval_step_interval': eval_step_interval,
    'validation_batch_size': validation_batch_size,
    'print_misclassified_test_images': print_misclassified_test_images,
    'output_graph': output_graph,
    'output_labels': output_labels
})

# Setting the FLAGS in retrain allows us to call the functions directly
retrain.FLAGS = FLAGS

This is one peice of the original project that I kept, which is incredibly useful. The original creator had a FLAGS table which contained the variables used throughout the other fucntions. I will definitely be doing this in the future using Jupyter Notebooks as it was very helpful for keeping things in one place.

## Main function

In [6]:
  # Setup the directory we'll write summaries to for TensorBoard
  if tf.gfile.Exists(FLAGS.summaries_dir):
    tf.gfile.DeleteRecursively(FLAGS.summaries_dir)
  tf.gfile.MakeDirs(FLAGS.summaries_dir)

  # Set up the pre-trained graph.
  graph, bottleneck_tensor, jpeg_data_tensor, resized_image_tensor = (
      retrain.create_inception_graph())

  # Look at the folder structure, and create lists of all the images.
  # This is why we use placeholder images when we reuse bottleneck files.
  image_lists = retrain.create_image_lists(stored_images_resized, 80, 10)
  class_count = len(image_lists.keys())
  if class_count == 0:
    raise Exception('No valid folders of images found at ' + FLAGS.image_dir)
  if class_count == 1:
    raise Exception('Only one valid folder of images found at ' + FLAGS.image_dir +
          ' - multiple classes are needed for classification.')

  with tf.Session(graph=graph) as sess:

    # Calculate and cache bottleneck files based on the resized images
    retrain.cache_bottlenecks(sess, image_lists, FLAGS.image_dir,
                    FLAGS.bottleneck_dir, jpeg_data_tensor,
                    bottleneck_tensor)

    # Add the new layer that we'll be training.
    (train_step, cross_entropy, bottleneck_input, ground_truth_input,
     final_tensor) = retrain.add_final_training_ops(len(image_lists.keys()),
                                            FLAGS.final_tensor_name,
                                            bottleneck_tensor)

    # Create the operations we need to evaluate the accuracy of our new layer.
    evaluation_step, prediction = retrain.add_evaluation_step(
        final_tensor, ground_truth_input)

    # Merge all the summaries and write them out to the summaries_dir
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(FLAGS.summaries_dir + '/train',
                                         sess.graph)

    validation_writer = tf.summary.FileWriter(
        FLAGS.summaries_dir + '/validation')

    # Set up all our weights to their initial default values.
    init = tf.global_variables_initializer()
    sess.run(init)
    
    # Save the original graph, so we can compare results later!
    output_graph_def = graph_util.convert_variables_to_constants(
        sess, graph.as_graph_def(), [final_tensor_name])
    with gfile.FastGFile(output_graph_orig, 'wb') as f:
        f.write(output_graph_def.SerializeToString())

    # Run the training!
    for i in range(FLAGS.how_many_training_steps):

      (train_bottlenecks, train_ground_truth, _) = retrain.get_random_cached_bottlenecks(
             sess, image_lists, FLAGS.train_batch_size, 'training',
             FLAGS.bottleneck_dir, FLAGS.image_dir, jpeg_data_tensor,
             bottleneck_tensor)
    
      # Feed the bottlenecks and ground truth into the graph, and run a training
      # step. Capture training summaries for TensorBoard with the `merged` op.
      train_summary, _ = sess.run(
          [merged, train_step],
          feed_dict={bottleneck_input: train_bottlenecks,
                     ground_truth_input: train_ground_truth})
      train_writer.add_summary(train_summary, i)

      # Every so often, print out how well the graph is training.
      is_last_step = (i + 1 == FLAGS.how_many_training_steps)
      if (i % FLAGS.eval_step_interval) == 0 or is_last_step:
        train_accuracy, cross_entropy_value = sess.run(
            [evaluation_step, cross_entropy],
            feed_dict={bottleneck_input: train_bottlenecks,
                       ground_truth_input: train_ground_truth})
        print('%s: Step %d: Train accuracy = %.1f%%' % (datetime.now(), i,
                                                        train_accuracy * 100))
        print('%s: Step %d: Cross entropy = %f' % (datetime.now(), i,
                                                   cross_entropy_value))
        validation_bottlenecks, validation_ground_truth, _ = (
            retrain.get_random_cached_bottlenecks(
                sess, image_lists, FLAGS.validation_batch_size, 'validation',
                FLAGS.bottleneck_dir, FLAGS.image_dir, jpeg_data_tensor,
                bottleneck_tensor))
        # Run a validation step and capture training summaries for TensorBoard
        # with the `merged` op.
        validation_summary, validation_accuracy = sess.run(
            [merged, evaluation_step],
            feed_dict={bottleneck_input: validation_bottlenecks,
                       ground_truth_input: validation_ground_truth})
        validation_writer.add_summary(validation_summary, i)
        print('%s: Step %d: Validation accuracy = %.1f%% (N=%d)' %
              (datetime.now(), i, validation_accuracy * 100,
               len(validation_bottlenecks)))

    # We've completed all our training, so run a final test evaluation on
    # some new images we haven't used before.
    test_bottlenecks, test_ground_truth, test_filenames = (
        retrain.get_random_cached_bottlenecks(sess, image_lists, FLAGS.test_batch_size,
                                      'testing', FLAGS.bottleneck_dir,
                                      FLAGS.image_dir, jpeg_data_tensor,
                                      bottleneck_tensor))
    test_accuracy, predictions = sess.run(
        [evaluation_step, prediction],
        feed_dict={bottleneck_input: test_bottlenecks,
                   ground_truth_input: test_ground_truth})
    print('Final test accuracy = %.1f%% (N=%d)' % (
        test_accuracy * 100, len(test_bottlenecks)))

    if FLAGS.print_misclassified_test_images:
      print('=== MISCLASSIFIED TEST IMAGES ===')
      total = 0
      for i, test_filename in enumerate(test_filenames):
        if predictions[i] != test_ground_truth[i].argmax():
          print('%70s  %s' % (test_filename, list(image_lists.keys())[predictions[i]]))
          total += 1
          print(total)

    # Write out the trained graph and labels with the weights stored as
    # constants.
    output_graph_def = graph_util.convert_variables_to_constants(
        sess, graph.as_graph_def(), [FLAGS.final_tensor_name])
    with gfile.FastGFile(FLAGS.output_graph, 'wb') as f:
      f.write(output_graph_def.SerializeToString())
    with gfile.FastGFile(FLAGS.output_labels, 'w') as f:
      f.write('\n'.join(image_lists.keys()) + '\n')

Looking for images in 'Acura'
Looking for images in 'Aston_Martin'
Looking for images in 'Audi'
Looking for images in 'Bentley'
Looking for images in 'BMW'
Looking for images in 'Bugatti'
Looking for images in 'Buick'
Looking for images in 'Cadillac'
Looking for images in 'Chevy'
Looking for images in 'Chrystler'
Looking for images in 'Daewoo'
Looking for images in 'Dodge'
Looking for images in 'Ferrari'
Looking for images in 'Fiat'
Looking for images in 'Fisker'
Looking for images in 'Ford'
Looking for images in 'Geo'
Looking for images in 'GMC'
Looking for images in 'Honda'
Looking for images in 'Hummer'
Looking for images in 'Hyundai'
Looking for images in 'Infiniti'
Looking for images in 'Isuzu'
Looking for images in 'Jaguar'
Looking for images in 'Jeep'
Looking for images in 'Lamborghini'
Looking for images in 'Land_Rover'
Looking for images in 'Lincoln'
Looking for images in 'Maybach'
Looking for images in 'Mazda'
Looking for images in 'Mclaren'
Looking for images in 'Mercedes_Be

2018-05-09 21:54:57.732789: Step 70: Train accuracy = 58.0%
2018-05-09 21:54:57.732884: Step 70: Cross entropy = 1.709345
2018-05-09 21:54:57.807184: Step 70: Validation accuracy = 23.0% (N=100)
2018-05-09 21:54:58.517781: Step 80: Train accuracy = 62.0%
2018-05-09 21:54:58.517882: Step 80: Cross entropy = 1.554101
2018-05-09 21:54:58.587216: Step 80: Validation accuracy = 24.0% (N=100)
2018-05-09 21:54:59.313085: Step 90: Train accuracy = 70.0%
2018-05-09 21:54:59.313186: Step 90: Cross entropy = 1.511660
2018-05-09 21:54:59.384537: Step 90: Validation accuracy = 22.0% (N=100)
2018-05-09 21:55:00.153461: Step 100: Train accuracy = 76.0%
2018-05-09 21:55:00.153569: Step 100: Cross entropy = 1.223099
2018-05-09 21:55:00.227718: Step 100: Validation accuracy = 31.0% (N=100)
2018-05-09 21:55:01.010840: Step 110: Train accuracy = 74.0%
2018-05-09 21:55:01.010939: Step 110: Cross entropy = 1.239083
2018-05-09 21:55:01.093067: Step 110: Validation accuracy = 35.0% (N=100)
2018-05-09 21:55:01

2018-05-09 21:55:31.910464: Step 490: Train accuracy = 89.0%
2018-05-09 21:55:31.910564: Step 490: Cross entropy = 0.532923
2018-05-09 21:55:31.987616: Step 490: Validation accuracy = 32.0% (N=100)
2018-05-09 21:55:32.749120: Step 500: Train accuracy = 94.0%
2018-05-09 21:55:32.749214: Step 500: Cross entropy = 0.516827
2018-05-09 21:55:32.818691: Step 500: Validation accuracy = 37.0% (N=100)
2018-05-09 21:55:33.571770: Step 510: Train accuracy = 89.0%
2018-05-09 21:55:33.571866: Step 510: Cross entropy = 0.554540
2018-05-09 21:55:33.647941: Step 510: Validation accuracy = 30.0% (N=100)
2018-05-09 21:55:34.390950: Step 520: Train accuracy = 91.0%
2018-05-09 21:55:34.391051: Step 520: Cross entropy = 0.515003
2018-05-09 21:55:34.462442: Step 520: Validation accuracy = 33.0% (N=100)
2018-05-09 21:55:35.191483: Step 530: Train accuracy = 90.0%
2018-05-09 21:55:35.191581: Step 530: Cross entropy = 0.558828
2018-05-09 21:55:35.263258: Step 530: Validation accuracy = 25.0% (N=100)
2018-05-09

INFO:tensorflow:Froze 2 variables.
Converted 2 variables to const ops.


You can see in the above results in the classifications that were missed, it shows which car image, and the prediction. Here is where you can see a lot of interesting things about this data. Integers to the left are total number of incorrect predictions.

# Results



# First run:
#### Training params
architecture = 'inception_v3'  
final_tensor_name = "final_result"  
how_many_training_steps = 500  
learning_rate = 0.08  
testing_percentage = 10  
validation_percentage = 10  
eval_step_interval = 10  
train_batch_size = 100  
test_batch_size = -1  
validation_batch_size = 100  
print_misclassified_test_images = True  
##### Final test accuracy = 24.1% (N=12910)  
##### 9805 misclassified images (out of 16100)  

# Second run:
#### Training params
architecture = 'inception_v3'  
final_tensor_name = "final_result"  
how_many_training_steps = 1000  
learning_rate = 0.1  
testing_percentage = 10  
validation_percentage = 20  
eval_step_interval = 10  
train_batch_size = 100  
test_batch_size = -1  
validation_batch_size = 100  
print_misclassified_test_images = True  
##### Final test accuracy = 28.1% (N=12910)
##### 9283 misclassified images (out of 12910)
##### 3627 correctly classified images

# Third run:
#### Training params
architecture = 'inception_v3'  
final_tensor_name = "final_result"  
how_many_training_steps = 1000  
learning_rate = 0.2  
testing_percentage = 10  
validation_percentage = 20  
eval_step_interval = 10  
train_batch_size = 100  
test_batch_size = -1  
validation_batch_size = 100  
print_misclassified_test_images = True  
##### Final test accuracy = 31.2% (N=12910)
##### 8879 misclassified images (out of 12910)
##### 4031 correctly classified images

# Best run:
#### Training params
architecture = 'inception_v3'  
final_tensor_name = "final_result"  
how_many_training_steps = 600  
learning_rate = 0.2  
testing_percentage = 50  
validation_percentage = 50  
eval_step_interval = 10  
train_batch_size = 100  
test_batch_size = 500  
validation_batch_size = 100  
print_misclassified_test_images = True  
##### Final test accuracy = 40.2% (N=500)
##### 290 misclassified images (out of 500)
##### 210 correctly classified images


The final train accuracy was consistently ~99% after more than 800 training steps.
The final test accuracy was anywhere from 30%-40%. This might seem low, but considering that there are **45** different classifications of car manufacturers, this is pretty good. I imagine that they'd all be pretty close to eachother, and the only thing seperating one car from another is very small.

In [7]:
# Test with the test_images subdirs
for graph in (output_graph_orig, output_graph):
    print("\nTesting with graph=%s\n" % graph)
    for subdir in ('V'):
        test_dir = os.path.join(test_images_dir, subdir)
        for f in os.listdir(test_dir):
            if f.endswith(JPEG_EXTENSIONS):
                tf.reset_default_graph()
                image = os.path.join(test_dir, f)
                print(image)
                %run image_retraining/label_image.py --image=$image --graph=$graph --labels=$output_labels


Testing with graph=output_graph_orig.pb

newTest/V/015866.jpg
bentley (score = 0.02280)
audi (score = 0.02280)
jaguar (score = 0.02276)
daewoo (score = 0.02271)
fiat (score = 0.02268)
newTest/V/015885.jpg
mazda (score = 0.02276)
fiat (score = 0.02273)
chrystler (score = 0.02261)
hyundai (score = 0.02260)
tesla (score = 0.02260)
newTest/V/015896.jpg
tesla (score = 0.02277)
mazda (score = 0.02267)
ferrari (score = 0.02263)
jaguar (score = 0.02260)
bentley (score = 0.02257)
newTest/V/015898.jpg
dodge (score = 0.02278)
porsche (score = 0.02274)
bentley (score = 0.02271)
audi (score = 0.02264)
jaguar (score = 0.02259)

Testing with graph=output_graph.pb

newTest/V/015866.jpg
honda (score = 0.22089)
nissan (score = 0.14257)
toyota (score = 0.10304)
suzuki (score = 0.09888)
cadillac (score = 0.07771)
newTest/V/015885.jpg
volkswagen (score = 0.15539)
volvo (score = 0.13933)
buick (score = 0.13528)
audi (score = 0.09478)
suzuki (score = 0.09172)
newTest/V/015896.jpg
scion (score = 0.17936)
hon

Here you can see the point spread for a few images before/after.

## Things that went wrong

I originally started this project with a different idea in mind, I was going to use the code which Dr. Anderson provided for a Convolutional neural network from lecture 21. I quickly learned that building and validating my own model was a much bigger task than I originally thought. I began to search for an already trained model, that did its original task fairly well, and was just going to adapt it to my data. This was kind of the hard part, was finding a network that was used to solve a task that was remotely close to mine.

After some searching, and recalling that Dr. Anderson warned us about pre-trained neural networks... I reluctantly found the only way to get this done on time was to use the Google Inception-V3 Transfer network with Tensorflow. Although this wasn't my ideal chioce, this was easier to do because I was able to use the knowledge of an already trained network that solved a previous problem, and fine tune it for my problem.

## Things I Thought were noteworthy
A couple things I thought were really interesting:
1. When the model got it's prediction wrong, it was interesting to note what was similar between the car image that was wrong, and what the model thought it was. Many times a Volvo was predicted to be a Volkswagen, a Chevy is a Dodge and visa versa. To me this shows the similarities in vehicle design are pretty apparent to this model, like Domestic vehicles, European vehicles, Exotic vehicles are always paired with eachother.

2. Related to above, there was never a Ferrari predicted to be anything but an exotic car. Ferrari was only ever predicted to be a Lamborghini, Tesla, Jaguar and Spyker.

3. The majority of cars that fell into a "grey area" (blocky, large cars) were predicted to be a Volkswagen

4. Volvo and Volkswagen were the strongest link when guessed incorrectly

## Conclusions
What I would like to do next time is figure out how to record a "second guess" for the model. It would be interesting to see for instance, the top two guesses for each vehicle. I would also like to record the occurances of which cars that were predicted wrong, and compare them to what the prediction was. Seeing as I was able to achive ~40% correct classification, I would assume that the network I used was not the best suit for the purpose I was trying to apply it towards.

I'm sure there is quite a few things that this model has learned that just doesn't apply to my car data, and that there is also many things that the model could benefit from if more car data was used in the original training.

Reading about the Inception-V3 model, it appears that it's trained on 1000 different classes, but it is more general objects. I wanted to use multiple different Transfer learning networks and compare them. Especially if I could find one specifically for cars, that would be cool.

I am a little bit satisfied with the Inception-V3 Transfer learning model. I originally thought it would do much better, but seeing as it was trained on general objects I am not suprised. If I chose to try and classify something else besides specific car manufactuerers, I would choose something that is a little more general. For example, is it a hotdog or not a hotdog? I feel like the dataset I chose was not very similar to the one the Inception-V3 was trained on. Unfortuneatly I didn't have time to test the various other architectures that Imagenet or Tensorflow might have available.

Last but not least, I would have loved to graph more data. The complexity of trying to figure this all out by creating a network from scratch was difficult. I abandoned the idea after much reluctance, because I had to get the ball rolling on actually classifing things! (I really wanted to have a network built FOR cars). Overall I learned alot from this assignment alone, and will do the best I can in the future to not always jump to the easy "pre-trained" networks because of how this model has proved to be "so-so" in terms of prediction. Thank you!

### Sources
3D Object Representations for Fine-Grained Categorization
Jonathan Krause, Michael Stark, Jia Deng, Li Fei-Fei
4th IEEE Workshop on 3D Representation and Recognition, at ICCV 2013 (3dRR-13). Sydney, Australia. Dec. 8, 2013.

Cars dataset from Jonathan Krause at Stanford, Computer Science
https://ai.stanford.edu/~jkrause/cars/car_dataset.html

PowerAI Transfer Learning GitHub example
https://github.com/IBM/powerai-transfer-learning?cm_sp=IBMCode-_-image-recognition-training-powerai-notebooks-_-Get-the-Code

Franck Barillaud's GitHub [Franck Barillaud](https://github.com/fbarilla) 


In [8]:
import io
from IPython.nbformat import current
import glob
nbfile = glob.glob('Cavey-Final.ipynb')
if len(nbfile) > 1:
    print('More than one ipynb file. Using the first one.  nbfile=', nbfile)
with io.open(nbfile[0], 'r', encoding='utf-8') as f:
    nb = current.read(f, 'json')
word_count = 0
for cell in nb.worksheets[0].cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print('Word count for file', nbfile[0], 'is', word_count)

Word count for file Cavey-Final.ipynb is 2211


/Users/Boulderc30/anaconda3/lib/python3.6/site-packages/IPython/nbformat.py:13: ShimWarning: The `IPython.nbformat` package has been deprecated since IPython 4.0. You should import from nbformat instead.
  "You should import from nbformat instead.", ShimWarning)
/Users/Boulderc30/anaconda3/lib/python3.6/site-packages/nbformat/current.py:19: UserWarning: nbformat.current is deprecated.

- use nbformat for read/write/validate public API
- use nbformat.vX directly to composing notebooks of a particular version

  """)
